In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# ACADEMY WINNING MOVIES ON IMDB SORTED DESC BOX OFFICE HTML
imdb_html = requests.get('https://www.imdb.com/search/title/?release_date=2010-01-01,&sort=boxoffice_gross_us,desc&count=250')
soup = BeautifulSoup(imdb_html.content, 'html.parser')

In [14]:
# Scraped through the html code to find the div with the movies
main_div_tag = soup.find('div', {'class':'lister-list'})

In [18]:
# returns all 250 movies per page
movies_per_page = main_div_tag.find_all('div', {'class':'lister-item mode-advanced'})

In [23]:
contents = []
for item in movies_per_page:
    contents.append(item.find('div',{'class':'lister-item-content'}))

In [25]:
contents

250

In [199]:
# for loop through each row in the table to retrieve the movie index number for imdb
imdb_popular_movies_indices = [row.a['href'].split('/')[2] for row in table_body_tag.find_all('tr')]

In [200]:
# API CALLS RETRIEVES list of 100 json files each representing one movie
pop_100_jsons = [requests.get(f"http://www.omdbapi.com/?i={i}&apikey=5a111996").json() for i in imdb_popular_movies_indices]

In [202]:
# Creating a pandas DataFrame and converting into CSV file
pd.DataFrame(pop_100_jsons).to_csv('imdb_100_popular_movies.csv')